## `type` hierarchy

In [1]:
class Foo:
    pass

In [2]:
x = Foo()

In [3]:
type(x)

__main__.Foo

In [8]:
type(Foo)

type

In [5]:
type(type)

type

<img src="class-chain.png" />

## Metaclasses are class `factories`

In [10]:
# Uncomment this
# help(type)

In [11]:
Foo = type('Foo', (), {})
Foo

__main__.Foo

In [13]:
x = Foo()

In [12]:
type(x)

__main__.Foo

In [14]:
type(Foo)

type

In [16]:
isinstance(Foo, type)

True

## Adding attributes

In [21]:
Person = type('Person', (), {'name': '', 'age': 0, 'gender': 'NA', 
                             '__str__': lambda x: '{}, age:{}, sex: {}'.format(x.name, x.age, x.gender)})

In [22]:
Person

__main__.Person

In [26]:
# Building an object and adding attributes
john = Person()

john.name='John'
john.age = 25
john.gender = 'M'
print(john)

John, age:25, sex: M


In [28]:
# Can't we initialize directly ?
john = Person(name='John', age=25, sex='M')

TypeError: object() takes no parameters

## Metaclass to the rescue

In [77]:
class PersonType(type):
    """ A type for Person classes """
    
    
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        # Dynamic assignment of methods
        x.__init__ = cls.__myinit__
        x.__str__ = cls.__mystr__
        x.hello = cls.hello
        return x

    def __myinit__(self, name=None, age=0, gender='NA'):
        self.name = name
        self.age = age
        self.gender = gender
        
    def __mystr__(self):
        return "{}, age: {}, sex: {}".format(self.name, self.age, self.gender)
    
    def hello(self):
        print(self)
        print("Hi - I am ", self.name)

In [78]:
class Person(metaclass=PersonType):
    pass

In [79]:
john = Person('John', 25, 'M')
print(john)
john.hello()

John, age: 25, sex: M
John, age: 25, sex: M
Hi - I am  John


## A better way - Use `__prepare__` special method on the meta-class

In [1]:
class PersonPreparedType(type):
    """ A type for Person classes """
    
    @classmethod
    def __prepare__(mcs, cls, bases, **kwargs):
        print('Metaclass=>',mcs, cls)
        return {'__init__': mcs.__myinit__, '__str__': mcs.__mystr__, 'hello': mcs.hello}

    def __myinit__(self, name=None, age=0, gender='NA'):
        self.name = name
        self.age = age
        self.gender = gender
        
    def __mystr__(self):
        return "{}, age: {}, sex: {}".format(self.name, self.age, self.gender)
    
    def hello(self):
        print('Calling on',self)
        print("Hi - I am ", self.name)

In [2]:
class Person(metaclass=PersonPreparedType):
    pass

Metaclass=> <class '__main__.PersonPreparedType'> Person


In [3]:
john = Person('John', 25, 'M')
print(john)
john.hello()

John, age: 25, sex: M
Calling on John, age: 25, sex: M
Hi - I am  John


## Metaclass Methods

Metaclasses rely on several magic methods so it's quite useful to know a bit more about them.

In [32]:
class Meta(type):
    
    def __init__(cls, *args, **kwargs):
        print('__init__:',cls)
        type.__init__(cls, *args)
        
    @classmethod
    def __prepare__(mcs, cls, bases, **kwargs):
        print('__prepare__:',mcs,cls)
        return {'__new__': mcs.my_new, 'hello': mcs.hello}
    
    def __call__(cls, *args, **kwargs):
        print('__call__:',cls, args)
        return type.__call__(cls, *args, **kwargs)
    
    def __new__(mcs, cls, bases=(), dct={}):
        print('mcs __new__:',mcs, cls, bases)
        print('mcs __new__ dict:', dct)
        return type.__new__(mcs, cls, bases, dct)
    
    @classmethod
    def my_new(mcs,cls,bases=(),dct={}):
        print('__new__:',mcs,cls)
        return object.__new__(cls)
    
    def hello(self):
        print('Hello World')

In [33]:
class C(metaclass=Meta):
    
    def __init__(self, x=100):
        print('C __init__',self)

__prepare__: <class '__main__.Meta'> C
mcs __new__: <class '__main__.Meta'> C ()
mcs __new__ dict: {'__new__': <bound method Meta.my_new of <class '__main__.Meta'>>, 'hello': <function Meta.hello at 0x7fd3cb78bbf8>, '__module__': '__main__', '__qualname__': 'C', '__init__': <function C.__init__ at 0x7fd3caf738c8>}
__init__: <class '__main__.C'>


In [35]:
c=C(200)

__call__: <class '__main__.C'> (200,)
__new__: <class '__main__.Meta'> <class '__main__.C'>
C __init__ <__main__.C object at 0x7fd3caf74d30>


### Method Calling Order

### Metaclass (Metaclass & Class creation)

1. First the metaclass's `__prepare__` method is called. This method is called before the class body is executed and it must return a dictionary-like object that's used as the local namespace for all the code from the class body.
2. Then the metaclass's `__new__` method is called. The dictionary returned by `__prepare__` above is passed to this method (as the last) argument.
3. At this point the metaclass's object is created. Then `__init__` (metclass's `__init__`) is called on it. Note that this takes the class as the argument. The class is `created` at this point.

### Class (Instance creation)

1. First class's `__call__` method is called. Any arguments to class creation is also passed her.
2. Next class's `__new__` method is called. This can be over-ridden to customize `instance creation`
3. Finally the instance's `__init__` is called (via the class) and the instance object returned.

## Class Creation

<img src="class-creation.png" />

## Instance Creation

<img src="instance-creation.png" />

1. Sub-classes inherit the `metaclass`.
2. Meta-class is available as the `__class__` attribute

In [37]:
class D(C):
    pass

__prepare__: <class '__main__.Meta'> D
mcs __new__: <class '__main__.Meta'> D (<class '__main__.C'>,)
mcs __new__ dict: {'__new__': <bound method Meta.my_new of <class '__main__.Meta'>>, 'hello': <function Meta.hello at 0x7fd3cb78bbf8>, '__module__': '__main__', '__qualname__': 'D'}
__init__: <class '__main__.D'>


In [38]:
print(C.__class__)
print(D.__class__)

<class '__main__.Meta'>
<class '__main__.Meta'>


#### Make sure you understand the following `truisms` 

In [39]:
print(isinstance(C, Meta))

True


In [40]:
print(issubclass(C, Meta))

False


In [41]:
print(issubclass(D,C))

True


In [42]:
print(isinstance(D,Meta))

True


In [43]:
print(issubclass(C, object))

True


### References

1. https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/
2. https://realpython.com/python-metaclasses/
3. https://www.python.org/dev/peps/pep-3115/